In [180]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.datasets import make_moons
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# 1. KMeans Featurizer

In [353]:
class KMeansFeaturizer:
    """Transforms numeric data into k-means cluster memberships.
    
    This transformer runs k-means on the input data and converts each data point
    into the id of the closest cluster. If a target variable is present, it is 
    scaled and included as input to k-means in order to derive clusters that
    obey the classification boundary as well as group similar points together.

    Parameters
    ----------
    k: integer, optional, default 100
        The number of clusters to group data into.

    target_scale: float, [0, infty], optional, default 5.0
        The scaling factor for the target variable. Set this to zero to ignore
        the target. For classification problems, larger `target_scale` values 
        will produce clusters that better respect the class boundary.

    random_state : integer or numpy.RandomState, optional
        This is passed to k-means as the generator used to initialize the 
        kmeans centers. If an integer is given, it fixes the seed. Defaults to 
        the global numpy random number generator.

    Attributes
    ----------
    cluster_centers_ : array, [k, n_features]
        Coordinates of cluster centers. n_features does count the target column.
    """

    def __init__(self, k=100, target_scale=5.0, random_state=None):
        self.k = k
        self.target_scale = target_scale
        self.random_state = random_state
        self.cluster_encoder = OneHotEncoder().fit(np.array(range(k)).reshape(-1,1))
        
    def fit(self, X, y=None):
        """Runs k-means on the input data and find centroids.

        If no target is given (`y` is None) then run vanilla k-means on input `X`. 

        If target `y` is given, then include the target (weighted by `target_scale`) 
        as an extra dimension for k-means clustering. In this case, run k-means 
        twice, first with the target, then an extra iteration without.

        After fitting, the attribute `cluster_centers_` are set to the k-means
        centroids in the input space represented by `X`.

        Parameters
        ----------
        X : array-like or sparse matrix, shape=(n_data_points, n_features)

        y : vector of length n_data_points, optional, default None
            If provided, will be weighted with `target_scale` and included in 
            k-means clustering as hint.
        """
        if y is None:
            # No target variable, just do plain k-means
            km_model = KMeans(n_clusters=self.k, 
                              n_init=20, 
                              random_state=self.random_state)
            km_model.fit(X)

            self.km_model = km_model
            self.cluster_centers_ = km_model.cluster_centers_
            return self

        # There is target information. Apply appropriate scaling and include
        # into input data to k-means            
        data_with_target = np.hstack((X, y[:,np.newaxis]*self.target_scale))
    
        # Build a pre-training k-means model on data and target
        km_model_pretrain = KMeans(n_clusters=self.k, 
                                   n_init=20, 
                                   random_state=self.random_state)
        km_model_pretrain.fit(data_with_target)

        # Run k-means a second time to get the clusters in the original space
        # without target info. Initialize using centroids found in pre-training.
        # Go through a single iteration of cluster assignment and centroid 
        # recomputation.
        
        km_model = KMeans(n_clusters=self.k, 
                          init=km_model_pretrain.cluster_centers_[:,:-1], # km_model_pretrain.cluster_centers_[:,:2], 
                          n_init=1, 
                          max_iter=1)
        km_model.fit(X)
        
        self.km_model = km_model
        self.cluster_centers_ = km_model.cluster_centers_
        return self
        
    def transform(self, X, y=None):
        """Outputs the closest cluster id for each input data point.

        Parameters
        ----------
        X : array-like or sparse matrix, shape=(n_data_points, n_features)

        y : vector of length n_data_points, optional, default None
            Target vector is ignored even if provided.

        Returns
        -------
        cluster_ids : array, shape[n_data_points,1]
        """
        clusters = self.km_model.predict(X)
        return self.cluster_encoder.transform(clusters.reshape(-1,1))
    
    def fit_transform(self, X, y=None):
        """Runs fit followed by transform.
        """
        self.fit(X, y)
        return self.transform(X, y)

# 2. Usage

## 2.1 Moons Data

### Sample Data

In [354]:
seed = 1
# train, test data
training_data, training_labels = make_moons(n_samples=2000, noise=0.2, random_state=seed)
test_data, test_labels = make_moons(n_samples=2000, noise=0.3, random_state=seed+5)

In [355]:
type(training_data), type(training_labels), training_data.shape, training_labels.shape

(numpy.ndarray, numpy.ndarray, (2000, 2), (2000,))

### Baseline

In [356]:
# model train
clf = LogisticRegression(random_state=seed)
clf.fit(training_data, training_labels)

LogisticRegression(random_state=1)

In [357]:
# test score
predictions = clf.predict_proba(test_data)[:,1]
roc_auc_score(test_labels, predictions)

0.9356779999999999

### KMeans Features

In [358]:
# kmeans featurizer init
kmf_hint = KMeansFeaturizer(k=5, target_scale=10, random_state=seed).fit(training_data, training_labels)

In [359]:
# kmeans feature for train
training_cluster_features = kmf_hint.transform(training_data)
training_cluster_features= training_cluster_features.todense()
training_data = np.hstack((training_data, training_cluster_features))

In [360]:
# model train
clf = LogisticRegression(random_state=seed)
clf.fit(training_data, training_labels)

LogisticRegression(random_state=1)

In [361]:
# kmeans feature for test 
test_cluster_features = kmf_hint.transform(test_data)
test_cluster_features= test_cluster_features.todense()
test_data = np.hstack((test_data, test_cluster_features))

In [362]:
# test score
predictions = clf.predict_proba(test_data)[:,1]
roc_auc_score(test_labels, predictions)

0.948393

## 2.2 Titanic Data

### Data

In [363]:
# Titanic data
df_titanic = (pd.read_csv('../data/titanic.csv')
              .astype({'Survived':'object', 'Pclass':'object'})
              .drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
              .replace({'Sex': {'male':0, 'female':1}})
              .pipe(lambda df: pd.concat([df, pd.get_dummies(df['Pclass'], prefix='Pclass')], axis=1)).drop(columns='Pclass')
              .pipe(lambda df: pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)).drop(columns='Embarked')
              .dropna()
             )

df_titanic_train, df_titanic_val = train_test_split(df_titanic, test_size=0.2)
x_train = df_titanic_train.drop(columns=['Survived']).values
y_train = np.array(df_titanic_train['Survived'].tolist())
x_val = df_titanic_val.drop(columns=['Survived']).values
y_val = np.array(df_titanic_val['Survived'].tolist())

In [364]:
type(x_train), type(y_train), x_train.shape, y_train.shape

(numpy.ndarray, numpy.ndarray, (571, 11), (571,))

### Baseline

In [366]:
# model train
clf = LogisticRegression(random_state=seed, max_iter=1000)
clf.fit(x_train, y_train)
print(x_train.shape)

(571, 11)


In [367]:
# test score
predictions = clf.predict_proba(x_val)[:,1]
roc_auc_score(y_val, predictions)

0.8077095472882115

### KMeans feature

In [382]:
kmf = KMeansFeaturizer(k=4, target_scale=10, random_state=seed).fit(x_train, y_train)

In [383]:
# kmeans feature for train
training_cluster_features = kmf.transform(x_train)

In [384]:
training_cluster_features= training_cluster_features.todense()
x_train = np.hstack((x_train, training_cluster_features))

In [385]:
# model train
clf = LogisticRegression(random_state=seed, max_iter=1000)
clf.fit(x_train, y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [386]:
# kmeans feature for test 
test_cluster_features = kmf.transform(x_val)
test_cluster_features= test_cluster_features.todense()
x_val = np.hstack((x_val, test_cluster_features))
print(x_val.shape)

(143, 45)


In [387]:
# test score
predictions = clf.predict_proba(x_val)[:,1]
auc_base = roc_auc_score(y_val, predictions)
print(f'baseline auc == {auc_base}')

baseline auc == 0.8099506947557149


### Optimize n_clusters

In [390]:
def k_test(k, x_train, y_train, x_val, y_val):
    # kmeans center init with x, y values
    kmf = KMeansFeaturizer(k=k, target_scale=10, random_state=seed).fit(x_train, y_train)
    
    # kmeans feature for train
    training_cluster_features = kmf.transform(x_train)

    training_cluster_features= training_cluster_features.todense()
    x_train = np.hstack((x_train, training_cluster_features))

    # model train
    clf = LogisticRegression(random_state=seed, max_iter=1000)
    clf.fit(x_train, y_train)

    # kmeans feature for test 
    test_cluster_features = kmf.transform(x_val)
    test_cluster_features= test_cluster_features.todense()
    x_val = np.hstack((x_val, test_cluster_features))

    # test score
    predictions = clf.predict_proba(x_val)[:,1]
    auc = roc_auc_score(y_val, predictions)
    
    return auc
    
n_clusters = list(range(2, 30))
auc_history = []
for n_cluster in n_clusters:
    auc = k_test(n_cluster, x_train, y_train, x_val, y_val)
    print(f'n_cluster == {n_cluster}, auc == {auc:.4f}')
    auc_history.append(auc)

n_cluster == 2, auc == 0.8113
n_cluster == 3, auc == 0.8088
n_cluster == 4, auc == 0.8104
n_cluster == 5, auc == 0.8097
n_cluster == 6, auc == 0.8100
n_cluster == 7, auc == 0.8073
n_cluster == 8, auc == 0.8021
n_cluster == 9, auc == 0.8086
n_cluster == 10, auc == 0.8079
n_cluster == 11, auc == 0.8048
n_cluster == 12, auc == 0.8061
n_cluster == 13, auc == 0.8079
n_cluster == 14, auc == 0.8084
n_cluster == 15, auc == 0.8048
n_cluster == 16, auc == 0.8048
n_cluster == 17, auc == 0.7972
n_cluster == 18, auc == 0.8019
n_cluster == 19, auc == 0.8008
n_cluster == 20, auc == 0.7965
n_cluster == 21, auc == 0.8037
n_cluster == 22, auc == 0.8075
n_cluster == 23, auc == 0.8055
n_cluster == 24, auc == 0.8084
n_cluster == 25, auc == 0.8082
n_cluster == 26, auc == 0.8082
n_cluster == 27, auc == 0.8030
n_cluster == 28, auc == 0.8061
n_cluster == 29, auc == 0.8064


In [389]:
n_clusters[np.argmax(auc_history)], np.max(auc_history)

(2, 0.811295383236217)

### Performance

+ baseline auc : 0.8077
+ kmeans featurizer auc : 0.8112(n_clusters == 2)